In [8]:
import pandas as pd
import os
import sqlite3
import django
from django.apps import apps

# Ez csak illusztrációnak, ne így töltsetek adatokat....


def setup_django():
    """
    A notebook-nak a Django project gyökerében kell lennie.
    a Django project gyökeréből is kell indítanod az ipython notebook-ot.
    Ez a függvény beállítja a Django környezetet.
    """
    try:
        if not apps.ready:
            os.environ.setdefault(
                "DJANGO_SETTINGS_MODULE", "base_template.settings"
            )  # beállítjuk h. melyik a settings file
            os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
            # Django biztonsági okból tiltja az async használatot bizonyos környezetekben, kivéve ha ezt engedélyezed.
            # Ha pl. notebookból, scriptből vagy Jupyterből hívsz be modelleket vagy ORM-et, akkor ez kell, különben hibát dob.
            django.setup()  # Ez a függvény elindítja a Django keretrendszert
            print("✅ Django environment successfully set up.")
        else:
            print("ℹ️ Django environment already set up.")
    except Exception as e:
        print(f"❌ Error during Django setup: {e}")

In [9]:
# Hívjuk meg és akkor ezután lehet nézegetni az ORM modelleket:
setup_django()

ℹ️ Django environment already set up.


In [10]:
from main.models import Friend
import pandas as pd

In [11]:
# Új rekord hozzáadás:
f1 = Friend(name="Kiss János", phone="+36301234567")
f1.save()

f1 = Friend(name="Zoli", phone="+421421421")
f1.save()

try:
    f1 = Friend(name="Zoli", phone="+421421421")
    f1.save()
except Exception as e:
    print(f"Ennek integrity error-t kell adnia: {e}")

f1 = Friend(name="Peti", phone="+42176576576")
f1.save()

Ennek integrity error-t kell adnia: UNIQUE constraint failed: main_friend.name, main_friend.phone


D:\programming\WebProg2025Fall\webprogenv\Lib\site-packages\django\db\models\fields\__init__.py:1671: RuntimeWarning: DateTimeField Friend.created_at received a naive datetime (2025-10-13 19:22:35.093246) while time zone support is active.
  warnings.warn(
D:\programming\WebProg2025Fall\webprogenv\Lib\site-packages\django\db\models\fields\__init__.py:1671: RuntimeWarning: DateTimeField Friend.created_at received a naive datetime (2025-10-13 19:22:35.102107) while time zone support is active.
  warnings.warn(
D:\programming\WebProg2025Fall\webprogenv\Lib\site-packages\django\db\models\fields\__init__.py:1671: RuntimeWarning: DateTimeField Friend.created_at received a naive datetime (2025-10-13 19:22:35.106668) while time zone support is active.
  warnings.warn(
D:\programming\WebProg2025Fall\webprogenv\Lib\site-packages\django\db\models\fields\__init__.py:1671: RuntimeWarning: DateTimeField Friend.created_at received a naive datetime (2025-10-13 19:22:35.107684) while time zone support 

In [17]:
# Egy filterezés példa:
friends_named_zoli = Friend.objects.all()
print(friends_named_zoli)
display(pd.DataFrame(list(friends_named_zoli)))

<QuerySet [<Friend: Zoli>, <Friend: Peti>, <Friend: Kiss János>]>


,0
0,Zoli
1,Peti
2,Kiss János


In [18]:
# Egy filterezés példa:
friends_named_zoli = Friend.objects.filter(name="Zoli").values()
print(friends_named_zoli)
display(pd.DataFrame(list(friends_named_zoli)))

<QuerySet [{'id': 7, 'created_at': datetime.datetime(2025, 10, 13, 19, 22, 35, 102107, tzinfo=datetime.timezone.utc), 'version_number': 1, 'name': 'Zoli', 'phone': '+421421421'}]>


,id,created_at,version_number,name,phone
0,7,2025-10-13 19:22:35.102107+00:00,1,Zoli,+421421421


In [14]:
friends = Friend.objects.filter(name__in=["Zoli", "Peti"]).values("name", "phone")
# name__in -> where name in (...)
# values('name', 'phone') -> select name, phone
# __lt, __lte, __gt, __gte hasonlóan működik mint az __in operator
# __contains -> like %...%
# __icontains -> like %...% case insensitive
# __startswith -> like ...%
# __endswith -> like %...
display(pd.DataFrame(list(friends)))

,name,phone
0,Zoli,+421421421
1,Peti,+42176576576


In [15]:
# Példa rekord törlésre:
Friend.objects.filter(name="Kiss János").delete()

friends_named_zoli = Friend.objects.filter(name="Zoli").values()
display(pd.DataFrame(list(friends_named_zoli)))

,id,created_at,version_number,name,phone
0,7,2025-10-13 19:22:35.102107+00:00,1,Zoli,+421421421
